# M01. Impute Inputs
- Normalizes model inputs
- Normalizes Steamer projections
- Uses Steamer projections to impute model inputs

In [1]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "U3. Classes.ipynb"
%run "D3. Simulation Functions.ipynb"

baseball_path = r'C:\Users\james\Documents\MLB\Database'

db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
engine = create_engine(f'sqlite:///{db_path}')

here
Done
Code was last run on: 2023-11-06


In [2]:
%run "A03. Steamer.ipynb"

### Batters

##### Dataset

In [3]:
# Choose the last instance of each player in each game, assuming they have enough PAs
sql_query = f'''
WITH ranked_data AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY gamePk, batter ORDER BY atBatIndex DESC) AS rn
  FROM "Dataset"
)
SELECT *
FROM ranked_data
WHERE pa_b >= 40 AND pa_b <= 300 AND rn = 1
'''

hitters_df = pd.read_sql_query(sql_query, con=engine)

In [4]:
# Standardize the data using StandardScaler
batter_stats_scaler = StandardScaler()
hitters_df[batter_stats] = batter_stats_scaler.fit_transform(hitters_df[batter_stats])

# Save the trained StandardScaler object
with open(os.path.join(model_path, "batter_stats_scaler_20231027.pkl"), "wb") as file:
    pickle.dump(batter_stats_scaler, file)

##### Steamer

In [5]:
# Choose the last instance of each player in each game, assuming they have enough PAs
sql_query = f'''
  SELECT *
  FROM "Steamer Hitters"
  WHERE "PA" >= 40
'''

steamer_hitters_df = pd.read_sql_query(sql_query, con=engine)

In [6]:
# Clean
steamer_hitters_df2 = clean_steamer_hitters(steamer_hitters_df)

In [7]:
# Standardize the data using StandardScaler
batter_stats_fg_scaler = StandardScaler()
steamer_hitters_df2[batter_stats_fg] = batter_stats_fg_scaler.fit_transform(steamer_hitters_df2[batter_stats_fg])

# Save the trained StandardScaler object
with open(os.path.join(model_path, "batter_stats_fg_scaler_20231027.pkl"), "wb") as file:
    pickle.dump(batter_stats_fg_scaler, file)

##### Merge

In [8]:
# Create column steamer_date column in hitters_df equal to the highest number <= a number in this list of uniques
steamer_dates = list(steamer_hitters_df2['date'].unique())

# Define a function to find the largest number in "steamer_dates" less than or equal to a given "date"
def find_steamer_date(date):
    max_steamer_date = max(filter(lambda d: d <= date, steamer_dates), default=None)
    return max_steamer_date

# Apply the function to create the "steamer_date" column in your DataFrame
hitters_df["steamer_date"] = hitters_df["date"].apply(find_steamer_date)

In [9]:
# Steamer stats we want to keep
batter_stats_fg_plus = ['mlbamid', 'steamerid', 'date'] + batter_stats_fg 
# Merge
hitters_merged_df = pd.merge(hitters_df, steamer_hitters_df2[batter_stats_fg_plus], left_on=['batter', 'steamer_date'], right_on=['mlbamid', 'date'], how='inner')
# Only keep one observation per player per game 
# Consider only keeping one observation per player per week/Steamer weekly projection
hitters_merged_df.drop_duplicates(['gamePk', 'batter'], inplace=True, keep='last')
# Only keep those without missing data
hitters_merged_df = hitters_merged_df.dropna(subset=batter_stats)

##### Impute

In [10]:
# Add hands to use in imputation
batter_stats_fg_imp = batter_stats_fg + ['b_L', 'p_L']

# Separate the features and target columns
features = hitters_merged_df[batter_stats_fg_imp]
target = hitters_merged_df[batter_stats]

# Create and fit the model
batter_imputations_model = keras.Sequential([
    keras.layers.Dense(25, activation='relu', input_shape=(len(batter_stats_fg_imp),)),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(len(batter_stats))  
    ])

# Compile the model
batter_imputations_model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
batter_imputations_model.fit(features, target, epochs=20, batch_size=32)

# Pickle
with open(os.path.join(model_path, "batter_imputations_model_20231027.pkl"), "wb") as file:
    pickle.dump(batter_imputations_model, file)

Epoch 1/20
10611/10611 [==============================] - 8s 717us/step - loss: 0.6458
Epoch 2/20
10611/10611 [==============================] - 8s 716us/step - loss: 0.6298
Epoch 3/20
10611/10611 [==============================] - 8s 777us/step - loss: 0.6269
Epoch 4/20
10611/10611 [==============================] - 9s 828us/step - loss: 0.6250
Epoch 5/20
10611/10611 [==============================] - 8s 769us/step - loss: 0.6234
Epoch 6/20
10611/10611 [==============================] - 8s 783us/step - loss: 0.6220
Epoch 7/20
10611/10611 [==============================] - 8s 711us/step - loss: 0.6210
Epoch 8/20
10611/10611 [==============================] - 7s 705us/step - loss: 0.6204
Epoch 9/20
10611/10611 [==============================] - 8s 716us/step - loss: 0.6198
Epoch 10/20
10611/10611 [==============================] - 8s 713us/step - loss: 0.6192
Epoch 11/20
10611/10611 [==============================] - 7s 698us/step - loss: 0.6187
Epoch 12/20
10611/10611 [================

In [11]:
# Use the trained model to make predictions
hitters_merged_df[batter_stats] = batter_imputations_model.predict(hitters_merged_df[batter_stats_fg_imp])

10611/10611 [==============================] - 5s 438us/step


### Pitchers

##### Dataset

In [12]:
# Select dataset
# Choose the last instance of each player in each game, assuming they have enough PAs
sql_query = f'''
WITH ranked_data AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY gamePk, pitcher ORDER BY atBatIndex DESC) AS rn
  FROM "Dataset"
)
SELECT *
FROM ranked_data
WHERE pa_p >= 40 AND pa_p <= 300 AND rn = 1
'''

pitchers_df = pd.read_sql_query(sql_query, con=engine)

In [13]:
# Standardize the data using StandardScaler
pitcher_stats_scaler = StandardScaler()
pitchers_df[pitcher_stats] = pitcher_stats_scaler.fit_transform(pitchers_df[pitcher_stats])

# Save the trained StandardScaler object
with open(os.path.join(model_path, "pitcher_stats_scaler_20231027.pkl"), "wb") as file:
    pickle.dump(pitcher_stats_scaler, file)

##### Steamer

In [14]:
# Choose the last instance of each player in each game, assuming they have enough PAs
sql_query = f'''
  SELECT *
  FROM "Steamer Pitchers"
  WHERE "PA" >= 40
'''

steamer_pitchers_df = pd.read_sql_query(sql_query, con=engine)

In [15]:
# Clean
steamer_pitchers_df2 = clean_steamer_pitchers(steamer_pitchers_df)

In [16]:
steamer_pitchers_df2.dropna(subset=pitcher_stats_fg2, inplace=True)

In [17]:
# Standardize the data using StandardScaler
pitcher_stats_fg_scaler = StandardScaler()
pitcher_stats_fg_scaled = pitcher_stats_fg_scaler.fit_transform(steamer_pitchers_df2[pitcher_stats_fg])
pitcher_stats_fg_scaled = pd.DataFrame(pitcher_stats_fg_scaled, columns=pitcher_stats_fg)

# Save the trained StandardScaler object
with open(os.path.join(model_path, "pitcher_stats_fg_scaler_20231027.pkl"), "wb") as file:
    pickle.dump(pitcher_stats_fg_scaler, file)

##### Merge

In [18]:
# Create column steamer_date column in hitters_df equal to the highest number <= a number in this list of uniques
steamer_dates = list(steamer_pitchers_df2['date'].unique())

# Define a function to find the largest number in "steamer_dates" less than or equal to a given "date"
def find_steamer_date(date):
    max_steamer_date = max(filter(lambda d: d <= date, steamer_dates), default=None)
    return max_steamer_date

# Apply the function to create the "steamer_date" column in your DataFrame
pitchers_df["steamer_date"] = pitchers_df["date"].apply(find_steamer_date)

In [19]:
# Steamer stats we want to keep
pitcher_stats_fg_plus = ['mlbamid', 'steamerid', 'date'] + pitcher_stats_fg2 
# Merge
pitchers_merged_df = pd.merge(pitchers_df, steamer_pitchers_df2[pitcher_stats_fg_plus], left_on=['pitcher', 'steamer_date'], right_on=['mlbamid', 'date'], how='inner')
# Only keep one observation per player per game 
# Consider only keeping one observation per player per week/Steamer weekly projection
pitchers_merged_df.drop_duplicates(['gamePk', 'pitcher'], inplace=True, keep='last')
# Only keep those without missing data
pitchers_merged_df = pitchers_merged_df.dropna(subset=pitcher_stats).dropna(subset=pitcher_stats_fg2)

##### Impute

In [20]:
# Add hands to use in imputation
pitcher_stats_fg_imp = pitcher_stats_fg + ['b_L', 'p_L']

# Separate the features and target columns
features = pitchers_merged_df[pitcher_stats_fg_imp]
target = pitchers_merged_df[pitcher_stats]

# Create and fit the model
pitcher_imputations_model = keras.Sequential([
    keras.layers.Dense(25, activation='relu', input_shape=(len(pitcher_stats_fg_imp),)),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(len(pitcher_stats))  
    ])

# Compile the model
pitcher_imputations_model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
pitcher_imputations_model.fit(features, target, epochs=20, batch_size=32)

# Pickle
with open(os.path.join(model_path, "pitcher_imputations_model_20231027.pkl"), "wb") as file:
    pickle.dump(pitcher_imputations_model, file)

Epoch 1/20
4991/4991 [==============================] - 4s 709us/step - loss: 0.8243
Epoch 2/20
4991/4991 [==============================] - 4s 716us/step - loss: 0.7824
Epoch 3/20
4991/4991 [==============================] - 4s 715us/step - loss: 0.7749
Epoch 4/20
4991/4991 [==============================] - 4s 716us/step - loss: 0.7715
Epoch 5/20
4991/4991 [==============================] - 4s 707us/step - loss: 0.7701
Epoch 6/20
4991/4991 [==============================] - 4s 709us/step - loss: 0.7690
Epoch 7/20
4991/4991 [==============================] - 4s 706us/step - loss: 0.7681
Epoch 8/20
4991/4991 [==============================] - 4s 711us/step - loss: 0.7673
Epoch 9/20
4991/4991 [==============================] - 4s 728us/step - loss: 0.7667
Epoch 10/20
4991/4991 [==============================] - 4s 710us/step - loss: 0.7662
Epoch 11/20
4991/4991 [==============================] - 4s 711us/step - loss: 0.7656
Epoch 12/20
4991/4991 [==============================] - 4s 713

In [21]:
# Use the trained model to make predictions
pitchers_merged_df[pitcher_stats] = pitcher_imputations_model.predict(pitchers_merged_df[pitcher_stats_fg_imp])

4991/4991 [==============================] - 2s 476us/step
